# Dataset formatting

I'll bring the dataset into a line-by-line format

In [8]:
from bs4 import BeautifulSoup
import os

if os.path.exists("./datasets/wit3_de_en/train.tags.de-en"):
  os.remove("./datasets/wit3_de_en/train.tags.de-en")

with open("./datasets/wit3_de_en/de-en/train.tags.de-en.de", 'r') as reader:
    xml_train_de = BeautifulSoup(reader.read())

transcripts_de = xml_train_de.find_all("transcript")

with open("./datasets/wit3_de_en/de-en/train.tags.de-en.en", 'r') as reader:
    xml_train_en = BeautifulSoup(reader.read())

transcripts_en = xml_train_en.find_all("transcript")

transcripts_de_en = zip(transcripts_de, transcripts_en)

formatted_train_text = []

for (transcript_de, transcript_en) in transcripts_de_en:
    transcript_de_lines = transcript_de.text.splitlines()
    transcript_en_lines = transcript_en.text.splitlines()
    formatted_transcript = map(lambda line_de, line_en: line_de + ' ' + line_en + '\n', transcript_de_lines, transcript_en_lines)
    formatted_train_text.extend(formatted_transcript)

with open("./datasets/wit3_de_en/train.tags.de-en", "x") as writer:
    writer.writelines(formatted_train_text)

In [9]:
if os.path.exists("./datasets/wit3_de_en/valid.tags.de-en"):
  os.remove("./datasets/wit3_de_en/valid.tags.de-en")

with open("./datasets/wit3_de_en/de-en/valid.tags.de-en.de", 'r') as reader:
    xml_train_de = BeautifulSoup(reader.read())

transcripts_de = xml_train_de.find_all("transcript")

with open("./datasets/wit3_de_en/de-en/valid.tags.de-en.en", 'r') as reader:
    xml_train_en = BeautifulSoup(reader.read())

transcripts_en = xml_train_en.find_all("transcript")

transcripts_de_en = zip(transcripts_de, transcripts_en)

formatted_train_text = []

for (transcript_de, transcript_en) in transcripts_de_en:
    transcript_de_lines = transcript_de.text.splitlines()
    transcript_en_lines = transcript_en.text.splitlines()
    formatted_transcript = map(lambda line_de, line_en: line_de + ' ' + line_en + '\n', transcript_de_lines, transcript_en_lines)
    formatted_train_text.extend(formatted_transcript)

with open("./datasets/wit3_de_en/valid.tags.de-en", "x") as writer:
    writer.writelines(formatted_train_text)

## Training a classifier

In [5]:
MODEL_PATH = "./improved-diffusion/diffusion_models/diff_roc_pad_rand128_transformer_lr0.0001_0.0_2000_sqrt_Lsimple_h128_s2_d0.1_sd101_xstart_e2e"
DATASET_TRAIN_PATH = "./datasets/wit3_de_en/train.tags.de-en.txt"
DATASET_VALID_PATH = "./datasets/wit3_de_en/valid.tags.de-en.txt"
EPOCH = 6
BATCH_SIZE = 10
GRADIENT_ACCUMULATION_STEPS = 1
PRETRAINED_MODEL ='bert-base-uncased'
EXPERIMENT = 'roc'
SEED = 101
TASK = 'translation'
NOTES = ''
APP = f"--init_emb {MODEL_PATH} --n_embd {16} --learned_emb yes"
BLOCK_SIZE = 100
MODEL_TYPE = 'gpt2'

folder_name = "classifier_models"


if not os.path.isdir(folder_name):
    os.mkdir(folder_name)


Model_FILE = EXPERIMENT + \
    '_e={}_b={}_m={}_{}_{}_{}'.format(
        EPOCH, BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS,
        PRETRAINED_MODEL, os.path.basename(DATASET_TRAIN_PATH), SEED, TASK)
Model_FILE = Model_FILE + f'_{NOTES}'
logging_dir = os.path.join(folder_name, 'runs', Model_FILE)
Model_FILE = os.path.join(folder_name, Model_FILE)
app = f" --train_file={DATASET_TRAIN_PATH} --validation_file {DATASET_VALID_PATH} " \
        f" --task {TASK}"
app += " " + APP




COMMANDLINE = f"python transformers/examples/pytorch/language-modeling/train_translation.py \
        --output_dir={Model_FILE} \
        --model_name_or_path={PRETRAINED_MODEL} \
        --tokenizer_name={PRETRAINED_MODEL} \
        --per_device_train_batch_size {BATCH_SIZE} \
        --per_device_eval_batch_size {BATCH_SIZE} \
        --save_steps 50000 \
        --num_train_epochs {EPOCH} \
        --do_train --eval_steps 10000 --evaluation_strategy steps \
        --do_eval --dataloader_num_workers 4 \
        --save_total_limit 1 \
        --overwrite_output_dir  \
        --logging_dir {logging_dir} \
        --block_size {BLOCK_SIZE}  \
        --disable_tqdm True --model_type {MODEL_TYPE} \
        --padding_mode none \
        --gradient_accumulation_steps {GRADIENT_ACCUMULATION_STEPS} " \
                f"--experiment {EXPERIMENT} --seed {SEED}"


COMMANDLINE += app

with open(Model_FILE + '.sh', 'w') as f:
    print(COMMANDLINE, file=f)

print(COMMANDLINE)
os.system(COMMANDLINE)

python transformers/examples/pytorch/language-modeling/train_translation.py         --output_dir=classifier_models/roc_e=6_b=10_m=bert-base-uncased_train.tags.de-en.txt_101_translation_         --model_name_or_path=bert-base-uncased         --tokenizer_name=bert-base-uncased         --per_device_train_batch_size 10         --per_device_eval_batch_size 10         --save_steps 50000         --num_train_epochs 6         --do_train --eval_steps 10000 --evaluation_strategy steps         --do_eval --dataloader_num_workers 4         --save_total_limit 1         --overwrite_output_dir          --logging_dir classifier_models/runs/roc_e=6_b=10_m=bert-base-uncased_train.tags.de-en.txt_101_translation_         --block_size 100          --disable_tqdm True --model_type gpt2         --padding_mode none         --gradient_accumulation_steps 1 --experiment roc --seed 101 --train_file=./datasets/wit3_de_en/train.tags.de-en.txt --validation_file ./datasets/wit3_de_en/valid.tags.de-en.txt  --task transl

[INFO|configuration_utils.py:652] 2022-11-16 12:57:59,085 >> loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/ydemirag/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
[INFO|configuration_utils.py:688] 2022-11-16 12:57:59,086 >> Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0.dev0"


initializing the tokenizer with small vocab
****************************************************************************************************
loading from dataset-specific vocab
132353

 Initializing the model from scratch 
****************************************************************************************************
11/16/2022 12:58:01 - WARNING - datasets.fingerprint - Parameter 'function'=<function main.<locals>.tokenize_function at 0x7faf1c85d5f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Running tokenizer on dataset:   0%|          | 0/67 [00:00<?, ?ba/s]

11/16/2022 12:58:02 - INFO - datasets.arrow_writer - Done writing 64909 examples in 10989634 bytes .
11/16/2022 12:58:02 - INFO - datasets.fingerprint - Parameter 'function'=<function main.<locals>.tokenize_function at 0x7faf1c85d5f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead.


...

{'train_runtime': 6294.7622, 'train_samples_per_second': 61.87, 'train_steps_per_second': 6.187, 'train_loss': 0.14429008824810946, 'epoch': 6.0}


[INFO|modeling_utils.py:1081] 2022-11-16 14:43:03,775 >> Model weights saved in classifier_models/roc_e=6_b=10_m=bert-base-uncased_train.tags.de-en.txt_101_translation_/pytorch_model.bin
[INFO|trainer.py:2385] 2022-11-16 14:43:03,780 >> ***** Running Evaluation *****
[INFO|trainer.py:2387] 2022-11-16 14:43:03,780 >>   Num examples = 66513
[INFO|trainer.py:2390] 2022-11-16 14:43:03,780 >>   Batch size = 10


***** train metrics *****
  epoch                    =        6.0
  train_loss               =     0.1443
  train_runtime            = 1:44:54.76
  train_samples            =      64909
  train_samples_per_second =      61.87
  train_steps_per_second   =      6.187
11/16/2022 14:43:03 - INFO - __main__ - *** Evaluate ***
{'eval_loss': 0.0784742459654808, 'eval_runtime': 196.7309, 'eval_samples_per_second': 338.091, 'eval_steps_per_second': 33.813, 'epoch': 6.0}
***** eval metrics *****
  epoch                   =        6.0
  eval_loss               =     0.0785
  eval_runtime            = 0:03:16.73
  eval_samples            =      66513
  eval_samples_per_second =    338.091
  eval_steps_per_second   =     33.813
  perplexity              =     1.0816


[INFO|modelcard.py:460] 2022-11-16 14:46:20,982 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}
wandb: Waiting for W&B process to finish... (success).
wandb: | 0.485 MB of 0.485 MB uploaded (0.000 MB deduped)
wandb: Run history:
wandb:                      eval/loss █▄▃▁
wandb:                   eval/runtime ▁▁▁█
wandb:        eval/samples_per_second ███▁
wandb:          eval/steps_per_second ███▁
wandb:                    train/epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:              train/global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:            train/learning_rate ████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
wandb:                     train/loss █▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/tra

0

In [2]:
import argparse
import os, json, sys
import stanza
import spacy_stanza
import numpy as np
import torch as th
from transformers import set_seed
import torch.distributed as dist
from improved_diffusion.rounding import rounding_func, load_models, load_tokenizer
from improved_diffusion.test_util import get_weights, denoised_fn_round
from functools import partial
from improved_diffusion import dist_util, logger
from improved_diffusion.script_util import (
    NUM_CLASSES,
    model_and_diffusion_defaults,
    create_model_and_diffusion,
    add_dict_to_argparser,
    args_to_dict,
)
from transformers import AutoModelForCausalLM
sys.path.insert(0, './improved-diffusion/scripts')
from infill_util import langevin_fn3, get_score, langevin_fn3_compose, langevin_fn1, langevin_fn4, langevin_fn_tree, langevin_fn_length
from spacy.lang.en import English

MODEL_PATH = "./improved-diffusion/diffusion_models/diff_roc_pad_rand128_transformer_lr0.0001_0.0_2000_sqrt_Lsimple_h128_s2_d0.1_sd101_xstart_e2e"

partial_seq = ['A kid friendly venue named Alimentum is located on the riverside .',
                       'Alimentum , situated by the river , is quite child friendly .']

emb_model, tokenizer = load_models(modality='roc', mode='random', model_name_or_path='', emb_dim=128, file=MODEL_PATH, )
model3 = get_weights(emb_model, {'emb_scale_factor': 1.0})
model3 = model3.cuda()


##model_control = Classifier_GPT2.from_pretrained('./classifier_models/roc_e=6_b=10_m=bert-base-uncased_train.tags.de-en.txt_101_translation_').cuda()

model = AutoModelForCausalLM.from_pretrained('./classifier_models/roc_e=6_b=10_m=bert-base-uncased_train.tags.de-en.txt_101_translation_')



loading from ./improved-diffusion/diffusion_models/diff_roc_pad_rand128_transformer_lr0.0001_0.0_2000_sqrt_Lsimple_h128_s2_d0.1_sd101_xstart_e2e/vocab.json
11043


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [11]:
partial_seq = ['A kid friendly venue named Alimentum is located on the riverside .',
                       'Alimentum , situated by the river , is quite child friendly .']

tokens2id = {v:k for k, v in tokenizer.items()}
todo_pad_token = -1
pad_token = tokens2id['PAD']
encoded_partial_seq = [th.LongTensor([tokens2id.get(x, tokens2id['UNK']) for x in seq.split()]).cuda() for seq in partial_seq]



embedding = model3(encoded_partial_seq[0])
embedding
input_embs = th.nn.Parameter(embedding)
model_out = model(input_embs)

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)